In [7]:
from numpy import loadtxt
import csv
import copy
import math
import time
from IPython.display import clear_output

In [11]:


# functions to find 1 or 2 distance away from something
def Hamming_distance_1(d):
    '''
    This function finds all bits one Hamming distance away from given bit d (as list of strings)
    
    Inputs:
            d: Given bit, e.g. ['0', '1', '0'] 
    
    Ourputs:
            distance_away: a list of list of all bits 1 Hamming distance away from d, eg. [['1', '1', '0'], ['0', '0', '0'],....]
    '''
    distance_away = []
    for i in range(0,len(d)):
        d_copy = copy.deepcopy(d)
        if d_copy[i]=='0':
            d_copy[i] = '1'
            distance_away.append(d_copy) 
        else:
            d_copy[i] = '0'
            distance_away.append(d_copy) 
            
    return distance_away

def Hamming_distance_2(d):
    '''
    This function finds all bits two Hamming distance away from given bit d (as list of strings)
    
    Inputs:
            d: Given bit
    Ourputs:
            distance_away: a list of list of all bits 2 Hamming distance away from d
    '''
    distance_away = []
    for i in range(0,len(d)-1):
        for j in range(i+1,len(d)):
            d_copy = copy.deepcopy(d)
            if d_copy[i]=='0':
                d_copy[i] = '1'
            else:
                d_copy[i] = '0'
            
            if d_copy[j]=='0':
                d_copy[j] = '1'
            else:
                d_copy[j] = '0'
            distance_away.append(d_copy)
            
    return distance_away

In [52]:
# reading the input file as a list
with open('clustering_big.txt') as f:
    reader = csv.reader(f, delimiter=" ")
    d = list(reader)
    
first_line = d.pop(0) # number of points, pop first line of file (which is the number of points)
n = int(first_line[0]) # number of vortecies
bit = int(first_line[1]) # number of bits for each vortex

# creating the hash table
start_time = time.time()

# finding all bits with 1 or 2 Hamming distance away from all vortecies (for each of the vertices)
# code_dict is a dictionary with bits as key and voortecies numbers as values
# e.g. code_dict['['0','1','1']'] = [3,8] means ['0','1','1'] is one or two distance away from votrex 3 and also vortex 8
vortex = 1
code_dict = {}
for i in d:
    zero_away = [i[0:bit]]
    one_away = Hamming_distance_1(i[0:bit])
    two_away = Hamming_distance_2(i[0:bit])
    all_one_two_away = one_away + two_away  + zero_away
    for j in all_one_two_away:
        try:
            code_dict[str(j)].append(vortex)
        except:
            code_dict[str(j)] = [vortex]
    vortex += 1
    
print("--- %s seconds ---" % (time.time() - start_time))



--- 820.8175451755524 seconds ---


In [53]:
start_time = time.time()

# initilized clusters
# key --> vortex (cluster) number, value --> a list with all vortececies in the cluster
Clusters = {}
for i in range(1,n+1):
    Clusters[str(i)] = [i]
    
# initilized leaders
# key --> vortex number, value --> leader of the vortex
Leader = {}
for i in range(1,n+1):
    Leader[str(i)] = i
    
vortex = 1
for i in d: # for all vortices (described as bits)
    try:
        # the vortices 1 or 2 Hamming distance away from i
        to_be_combined = code_dict[str(i[0:bit])]
        for j in to_be_combined:
            # leader of j
            leader_j = copy.deepcopy(Leader[str(j)])
            # transfer the vortex in the smaler cluster to the bigger cluster (less leader update)
            if len(Clusters[str(Leader[str(vortex)])]) <= len(Clusters[str(leader_j)]):
                if leader_j != Leader[str(vortex)]: 
                    # put everything that is with current vortex in to the cluster of leader_j
                    Clusters[str(leader_j)] = Clusters[str(Leader[str(vortex)])] + Clusters[str(leader_j)]
                    # keep leader of current vortex (before updating it)
                    a = copy.deepcopy((Leader[str(vortex)]))
                    # update leader of every thing that was with current vortex in the same cluster (as they are transfered)
                    for k in Clusters[str(Leader[str(vortex)])]:
                        Leader[str(k)] = leader_j  
                    # delete things in the cluster of current vortex (they are trasfered to the  Clusters[str(leader_j)])
                    Clusters[str(a)] = [] 
            else:
                # leader of j
                leader_j = copy.deepcopy(Leader[str(j)])
                # update leader of every thing that was with j in the same cluster
                if leader_j != Leader[str(vortex)]:
                    # update leader of every thing that was with leader_j in the same cluster (as they are going to be transfered)
                    for k in Clusters[str(leader_j)]:
                        Leader[str(k)] = Leader[str(vortex)]  
                    Clusters[str(Leader[str(vortex)])] = Clusters[str(Leader[str(vortex)])] + Clusters[str(leader_j)]
                    # delete things in the cluster of leader_j (they are trasfered to the  Clusters[str(Leader[str(vortex)])])
                    Clusters[str(leader_j)] = []
            
    except:
        pass
    vortex += 1

# count unique values (leaders) in Clusters
unique_leaders = sum([1 if len(y)!=0 else 0 for (x,y) in Clusters.items()])

print("--- %s seconds ---" % (time.time() - start_time))

# print(Clusters)
print(unique_leaders)
# print(len(unique_leaders))

--- 86.36125206947327 seconds ---
6118
